In [ ]:
# Kaggle / Colab setup commands
# These shell commands clone the repository, move files into the working directory,
# upgrade pip, and install Python dependencies listed in requirements.txt.
# Execute this cell in a notebook environment (the `!` prefix runs shell commands).

# Clone the ADIS repository (adjust URL if using a fork)
!git clone https://github.com/sathishkumar67/ADIS.git

# Move repository contents into the notebook working directory (adjust paths for your environment)
!mv /kaggle/working/ADIS/* /kaggle/working/ # change this path to move files to the current directory

# Upgrade pip to the latest version
!pip install --upgrade pip

# Install required Python packages from the repository's requirements file
!pip install -r requirements.txt

In [ ]:
# Python imports used by the tuning notebook
# - joblib: save/load Optuna studies
# - optuna: hyperparameter optimization framework
# - hf_hub_download: download artifacts from Hugging Face Hub
# - unzip_file: local util that extracts archives with a progress bar
# - YOLO11Model: project model wrapper that provides .train() and .score
from __future__ import annotations
import os
import joblib
import optuna
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

In [ ]:
# Global dataset and experiment paths / names
# Update these variables if your local layout or dataset naming differs.
REPO_ID = "pt-sk/ADIS"  # HF repo id where the dataset zip may be stored
DATASET_NAME = "balanced_dataset"  # dataset root folder name (without .zip)
FILENAME_IN_REPO = f"{DATASET_NAME}.zip"
LOCAL_DIR = os.getcwd()
TRAIN_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/train"
VAL_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/val"
TEST_PATH = f"{LOCAL_DIR}/{DATASET_NAME}/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10
# Class labels in the dataset. Order matters — they should match dataset label IDs.
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake']
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"

# Path to pretrained weights (leave empty to use pretrained=True semantics in training)
# Example values: 'yolo11m.pt', 'yolo11s.pt', 'yolo11n.pt'
MODEL_PATH = ""

In [ ]:
# Download dataset from Hugging Face Hub (if hosted there) and unzip locally.
# The helper `unzip_file` provides a progress bar while extracting.

hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)

# Extract archive contents to LOCAL_DIR
unzip_file(DATASET_PATH, LOCAL_DIR)

# Optionally remove the downloaded archive to free disk space
os.remove(f"{LOCAL_DIR}/{DATASET_NAME}.zip")
print("deleted dataset.zip")

# Query number of CPU cores to set dataloader workers accordingly
num_cores = os.cpu_count()
print(f"No of Cores Available: {num_cores}")

In [ ]:
# Compose and write the dataset YAML file used by YOLO training code.
# The YAML follows the standard Ultralytics format with train/val/test paths, nc and names.

data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# Persist the YAML so model/trainer utilities can read it during training/validation
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    print("data yaml file written!.............")

In [ ]:
# Define Optuna objective for hyperparameter tuning.
# The objective creates a model, trains it with suggested hyperparameters and returns
# a scalar objective (validation mAP) for Optuna to maximize.

def objective(trial):
    """
    Optuna objective function.

    This function is called by Optuna for each trial. It should:
      - Sample hyperparameters via trial.suggest_*
      - Optionally register callbacks (e.g., to report intermediate results)
      - Instantiate the YOLO11Model and call .train(...) with hyperparameters
      - Return model.score (validation mAP or other metric used as objective)
    """

    # Callback to report intermediate results back to Optuna and support pruning
    def on_train_epoch_end(score, epoch):
        trial.report(score, step=epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    callbacks = {
        "on_train_epoch_end": on_train_epoch_end,
    }

    # Hyperparameter search space (tweak ranges to your needs)
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-2, log=True)
    lrf = trial.suggest_float("lrf", 0.1, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-2, log=True)
    warmup_momentum = trial.suggest_float("warmup_momentum", 0.0, 0.9)
    momentum = trial.suggest_float("momentum", 0.8, 0.99)

    # Training config passed to YOLO11Model.train. Values may be overridden by explicit args
    CONFIG_DICT = {
        "task": "detect",
        "mode": "train",
        "bohb": True,
        "custom_callbacks": callbacks,
        "data": DATA_YAML_FILE,
        "batch": 80,  # adjust based on GPU memory
        "imgsz": 320,
        "save": True,
        "device": 0,
        "workers": num_cores,
        "pretrained": True,
        "optimizer": "AdamW",
        "seed": 42,
        "epochs": 60,
        "warmup_epochs": 5,
        "patience": 5,
    }

    # Instantiate and train the model. The model should expose .score after training.
    model = YOLO11Model(MODEL_PATH)
    model.train(**CONFIG_DICT, lr0=lr0, lrf=lrf, momentum=momentum, weight_decay=weight_decay, warmup_momentum=warmup_momentum)

    # Return the metric Optuna should maximize (model.score is set by the training loop)
    return model.score

In [ ]:
# Run Optuna study to optimize hyperparameters and persist the study to disk.
# Adjust NUM_TRIALS to control search budget.
import joblib
NUM_TRIALS = 20  # number of trials to run

# Create or load a study configured to maximize the validation metric
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name="yolo11_tuning",
    load_if_exists=True,
)

# Run the optimization loop. This will call the objective function multiple times.
study.optimize(objective, n_trials=NUM_TRIALS)

# Optionally persist the study to disk so tuning progress can be resumed later
study_save_path = ""  # set to a path like 'study_yolo11.pkl' to save
if study_save_path:
    joblib.dump(study, study_save_path)
